In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from tqdm import tqdm
import re
import warnings
import gc
import scipy as sp
from scipy.stats import norm
from scipy.stats import spearmanr, kendalltau
from IPython.display import clear_output


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
players = pd.DataFrame.from_dict(pd.read_pickle('players.pkl'), orient='index').reset_index()

In [3]:
results = pd.read_pickle('results.pkl')

In [4]:
tournaments = pd.DataFrame.from_dict(pd.read_pickle('tournaments.pkl'), orient='index').reset_index()

## Навигация: <a class="anchor" id="toc"></a>
[0. Введение](#introduction)\
[1. BaseLine модель](#baseline)\
[2. EM Алгоритм](#em)\
[3. EM алгоритм v.2](#em2)\
[4. Выводы](#conclusions)

<a class="anchor" id="introduction"></a>
[К оглавлению](#toc)

# 0. Введение.

### Что я знаю о ЧГК?
Никогда не играл в спортивную версию ЧГК, но частенько играл в квизы и принимал участие в организации, что по сути есть развлекательная версия ЧГК для компаний. Смысл тот же: участники, команды, вопросы по турам, турнирная таблица. Разница существенная в том, что квиз больше развлекательная игра и там чаще вопросы одноходовки или вообще угадай мелодию или фильм по кадрам, ЧГК в этом плане серьезное мероприятие.\
По квизам тенденция такая, что команды, играющие хорошо, играют хорошо всегда, и чаще всего в командах есть костяк, к которому периодически присоединяются новички, но игры выигрывает в основном костяк. Новички могут внести сумбур в слаженную игру костяка, но чаще всего после этого они не играют уже. На квизах часто выигрывают команды, участники которой отвечают за свою конкретную сферу знайний, инженер, киноман, меломан и так далее. Поэтому на вопросах про кино "топит" именно киноман и так далее. В ЧГК, думаю, у хороших команд скорее всего аналогичная тенденция, человек не может знать все и хорошо одновременно.\
Сами квизы, если это одни и те же организаторы, редко отличаются по сложности от игры к игре, скорее стараются разнообразить саму игру, разбивая их сложности туров. Такого разнообразия по турнирам как в ЧГК тут нет. Но если взять разные квизы, то тут есть вариантивность по сложности. Некоторые квизы заточены под логику, другие под знания, тертьи на "поугарать". И чаще всего, команда, идущая постоянно в топе на "логических" квизах, не очень уверенно выступает на квизах по знаниям(поэтому я не люблю КвизПлиз - чаще всего или знаешь, или пошел я). Это я к тому, что как в квизе, как и в ЧГК, существует разбивка по сложности в зависимости от типа игры.

### Что я узнал о ЧГК?
Количество человек в команде ограничено 6 сверху, хотя есть случаи когда в турнирах принимали участие команды с большим фактическим числом игроков, но "за столом" принимать участия могут не более 6.\
У каждого игрока есть свой статус и рейтинг. Игрок может быть, например, капитаном, или легионером. Число легионеров в составе команды негативно сказывается на ее рейтинге по итогам игры.\
Команды зачастую имеют четкий базовый состав, могут менять состав и названия для каждой игры, но высокорейтинговые команды редко этим занимаются.
Турниры отличаются не только по своей сложности но и формату, продолжительности, охвату географии, числу команд и вопросов.

### Что содержится в данных?
Есть 3 файла с данными. 
1. __Игроки.__ Тут данные самих иргоков, ничего полезного я тут не увидел. Имена, фамилии, рейтинг, который никак особо к модели не прикрутить.
2. __Турниры.__ Тут полезной информации больше. Есть id турнира, для сопоставления с другими таблицами. Есть дата начала и окончания турнира, есть информация о числе туров и числе вопросов в каждом туре.
3. __Рузльтаты.__ Самые полезные данные. Тут куча информации по турнирам и командам, которые приняли участие в конкретном турнире. Рузльтаты команды, ее повопросные ответы(1/0), число правильных ответов и занятое место по итогам турнира. Для каждой комнады указан город, основное название и название для текущего турнира. Так же есть тип команды(это может быть, например, школьная или студенческая комнада). Для каждой команды на конкретном турнире указан состав, с метками о статусе игрока. 

### Что я сделал с данными?
Для начала, я агрегировал 2 таблицы: турниры и результаты. Из них взял полезную, на мой взгляд, информацию:
 - id турнира
 - id команды
 - ids игроков
 - тип турнира
 - продолжительность
 - информация по турам на турнире
 - число комнад на турнире
 - меняла ли комнада название
 - различные города команд
 - тип команды
 - итоговая позиция команды
 - ответы команды
 - число игроков в команде
 - статусы игроков
 
В дальнейшем некоторые существующие текстовые данные преобразовал в численные, по типу коэффициентов, например, для легионера - 0.85, а для капитана - 1.\
Немного почистил данные: есть игра, в которой приняли участие 4 команды, но ответов не давали, удалил.
Разбил данные поигрокам.\
Добавил коэффициент участия игрока в турнирах: отношения числа турниров, в которых принял участие игрок, к среднему по выборке.\
Добавил коэффициент смены команд для игрока: число команда, за которых игрок выступал, к среднему по выборке.\
В итоге датасет получился внушительным и едва помещался в память: 20млн строк и 18 столбцов.

[К оглавлению](#toc)\
[1. BaseLine модель](#baseline)

In [5]:
ids_train = tournaments.loc[(tournaments.dateStart>="2019-01-01T00:00:00+04:00") & (tournaments.dateStart<"2020-01-01T00:00:00+04:00"), 'id'].to_numpy()
ids_test = tournaments.loc[tournaments.dateStart>="2020-01-01T00:00:00+04:00", 'id'].to_numpy()

In [6]:
def create_tournaments_table(results_table, tournament_table, ids):
    rows = {'id_tournament': [], 
            'type_of_tournament': [], 
            'duration': [], 
            'questionQty': [], 
            'count_teams': [], 
            'id_team': [], 
            'factor_name': [], 
            'locations': [], 
            'flag_team': [], 
            'position': [], 
            'answers': [], 
            'count_players': [], 
            'ids_players': [],
            'flags_players': []}
    for _id in tqdm(ids):
        count_teams = len(results_table[_id])
        for team in results_table[_id]:
            if 'mask' in team.keys() and team['mask']:
                type_of_tournament = tournament_table.loc[tournament_table.id==_id, 'type'].values[0]['name']
                rows['id_tournament'].append(_id)
                rows['type_of_tournament'].append(type_of_tournament)
                
                date_end = pd.to_datetime(tournament_table.loc[tournament_table.id==_id, 'dateEnd'])
                date_start = pd.to_datetime(tournament_table.loc[tournament_table.id==_id, 'dateStart'])

                duration_ = (date_end - date_start).apply(lambda x: x.days*24*60+x.seconds//60).values[0]/60
                
                rows['duration'].append(duration_)
                
                question_qty = tournament_table.loc[tournament_table.id==_id, 'questionQty'].to_dict()
                rows['questionQty'].append(question_qty)
                rows['count_teams'].append(count_teams)
                
                rows['id_team'].append(team['team']['id'])
                
                name = team['team']['name']
                current_name = team['current']['name']
                if name == current_name:
                    rows['factor_name'].append('П')
                else:
                    rows['factor_name'].append('В')  
                
                if team['current']['town']:
                    rows['locations'].append(team['current']['town']['name'])
                else:
                    rows['locations'].append('')

                if team['flags']:
                    rows['flag_team'].append(team['flags'][0]['shortName'])
                else:
                    rows['flag_team'].append('')

                    
                rows['position'].append(team['position'])
                rows['answers'].append(team['mask'])          
                rows['count_players'].append(len(team['teamMembers']))    
                                
                ids_players = []
                flags_players = []
                
                for player in team['teamMembers']:
                    ids_players.append(str(player['player']['id']))
                    if player['flag']:
                        flags_players.append(player['flag'])
                    else:
                        flags_players.append('Н')
                                       
                rows['ids_players'].append(ids_players)  
                rows['flags_players'].append(flags_players)  
                
            else:
                break
    return rows
          

In [7]:
teams_results_train = pd.DataFrame.from_dict(create_tournaments_table(results, tournaments, ids_train))
teams_results_test = pd.DataFrame.from_dict(create_tournaments_table(results, tournaments, ids_test))

100%|██████████| 422/422 [00:46<00:00,  9.09it/s]


In [8]:
def clear_bad_questions(df):
    for id_t in tqdm(df['id_tournament'].unique()):
        indexes_bad_questions = []
        answers = []
        for answer in df[df.id_tournament==id_t]['answers']:
            for i in re.finditer(r'X|\?', answer):
                indexes_bad_questions.append(i.start())
        indexes_bad_questions = sorted(list(set(indexes_bad_questions)))
        if indexes_bad_questions:
            for answer in df[df.id_tournament==id_t]['answers']:
                answer = list(answer)
                for idx in indexes_bad_questions:
                    if idx < len(answer):
                        answer[idx] = ''
                answer = ''.join(answer)
                answers.append(answer)
            df.loc[df.id_tournament==id_t, 'answers'] = np.array(answers)
    return df

In [9]:
teams_results_train = clear_bad_questions(teams_results_train)
teams_results_test = clear_bad_questions(teams_results_test)

100%|██████████| 173/173 [00:00<00:00, 847.65it/s]


In [10]:
def temp_type_of_tournament_transform(x):
    if x == 'Обычный':
        return 1.
    elif x == 'Строго синхронный':
        return 2/3
    else:
        return .5

teams_results_train.type_of_tournament = teams_results_train.type_of_tournament.apply(temp_type_of_tournament_transform)
teams_results_test.type_of_tournament = teams_results_test.type_of_tournament.apply(temp_type_of_tournament_transform)

In [11]:
def temp_factor_name_transform(x):
    if x == 'П':
        return 1.
    else:
        return .9

teams_results_train.factor_name = teams_results_train.factor_name.apply(temp_factor_name_transform)
teams_results_test.factor_name = teams_results_test.factor_name.apply(temp_factor_name_transform)

In [12]:
def temp_flags_players_transform(x):
    count_legionnaire = x.count('Л')
    count_players = len(x)
    if count_legionnaire >= 2:
        return 1 - (1/count_players) - (1/count_players)*(count_legionnaire-2)
    else:
        return 1

teams_results_train['flags_players_legionnaire'] = teams_results_train.flags_players.apply(temp_flags_players_transform)
teams_results_test['flags_players_legionnaire'] = teams_results_test.flags_players.apply(temp_flags_players_transform)

In [13]:
def temp_flag_team_transform(x):
    r_factor = ['С', 'Ш', 'СтШ', 'СрШ', 'МШ', 'МлШ', 'Вуз', '!', 'Мол', 'ЛЛ']
    if x in r_factor:
        return 0.9
    else:
        return 1

teams_results_train['flag_team'] = teams_results_train.flag_team.apply(temp_flag_team_transform)
teams_results_test['flag_team'] = teams_results_test.flag_team.apply(temp_flag_team_transform)

In [14]:
li_train = zip(teams_results_train.ids_players, teams_results_train.flags_players)
teams_results_train['id_player'] = np.array([['-'.join(z) for z in list(zip(i, j))] for i, j in li_train])
teams_results_train = teams_results_train.drop(columns=['ids_players', 'flags_players'])

li_test = zip(teams_results_test.ids_players, teams_results_test.flags_players)
teams_results_test['id_player'] = np.array([['-'.join(z) for z in list(zip(i, j))] for i, j in li_test])
teams_results_test = teams_results_test.drop(columns=['ids_players', 'flags_players'])

In [15]:
rows = []
_ = teams_results_train.apply(lambda row: [rows.append([row['id_tournament'], 
                                                        row['type_of_tournament'], 
                                                        row['duration'], 
                                                        row['questionQty'], 
                                                        row['count_teams'], 
                                                        row['id_team'], 
                                                        row['factor_name'],
                                                        row['locations'], 
                                                        row['flag_team'], 
                                                        row['position'], 
                                                        row['answers'],
                                                        row['count_players'], 
                                                        row['flags_players_legionnaire'], 
                                                        ids]) 
                         for ids in row.id_player], axis=1)
teams_results_train = pd.DataFrame(rows, columns=teams_results_train.columns)
teams_results_train = teams_results_train.reset_index()

In [16]:
rows = []
_ = teams_results_test.apply(lambda row: [rows.append([row['id_tournament'], 
                                                        row['type_of_tournament'], 
                                                        row['duration'], 
                                                        row['questionQty'], 
                                                        row['count_teams'], 
                                                        row['id_team'], 
                                                        row['factor_name'],
                                                        row['locations'], 
                                                        row['flag_team'], 
                                                        row['position'], 
                                                        row['answers'],
                                                        row['count_players'], 
                                                        row['flags_players_legionnaire'], 
                                                        ids]) 
                         for ids in row.id_player], axis=1)
teams_results_test = pd.DataFrame(rows, columns=teams_results_test.columns)
teams_results_test = teams_results_test.reset_index()

In [17]:
def cut_flag(x):
    flag = x[-1]
    
    if flag=='К':
        return 1
    elif flag=='Б':
        return 0.95
    elif flag=='Н':
        return 0.9
    elif flag=='Л':
        return 0.85
    
teams_results_train['flag_player'] = teams_results_train.id_player.apply(cut_flag)
teams_results_train['id_player'] = teams_results_train.id_player.apply(lambda x: x[:-2:])

teams_results_test['flag_player'] = teams_results_test.id_player.apply(cut_flag)
teams_results_test['id_player'] = teams_results_test.id_player.apply(lambda x: x[:-2:])

In [18]:
rate_player_train = teams_results_train.groupby(by='id_player', 
                                                sort=False)['id_tournament', 'id_team'].agg(lambda x: len(list(set(x))))
rate_player_test = teams_results_test.groupby(by='id_player', 
                                              sort=False)['id_tournament', 'id_team'].agg(lambda x: len(list(set(x))))

In [19]:
mean_tournament_count = rate_player_train.loc[rate_player_train.id_tournament>3, 'id_tournament'].mean()

In [20]:
def tournament_rate_create(x, mean_tournament_count):
    if x<4:
        return 3/mean_tournament_count
    else:
        return x/mean_tournament_count

In [21]:
rate_player_train['tournament_rate'] = rate_player_train.id_tournament.apply(lambda x: tournament_rate_create(x, mean_tournament_count))
rate_player_test['tournament_rate'] = rate_player_test.id_tournament.apply(lambda x: tournament_rate_create(x, mean_tournament_count))

In [22]:
mean_team_count = rate_player_train.loc[rate_player_train.id_team>1, 'id_team'].mean()

In [23]:
rate_player_train['team_rate'] = rate_player_train.id_team.apply(lambda x: x/mean_team_count)
rate_player_test['team_rate'] = rate_player_test.id_team.apply(lambda x: x/mean_team_count)

In [24]:
teams_results_train = pd.merge(teams_results_train, rate_player_train[['tournament_rate', 'team_rate']], on=['id_player'])
teams_results_test = pd.merge(teams_results_test, rate_player_test[['tournament_rate', 'team_rate']], on=['id_player'])

In [25]:
def create_various_locations(df):
    for id_tournament in tqdm(df.id_tournament.unique()):
        various_locations = len(list(set(df.loc[df.id_tournament==id_tournament, 'locations'])))
        df.loc[df.id_tournament==id_tournament, 'various_locations'] = various_locations
    return df

In [26]:
teams_results_train = create_various_locations(teams_results_train)
teams_results_test = create_various_locations(teams_results_test)

100%|██████████| 173/173 [00:00<00:00, 498.52it/s]


In [27]:
teams_results_train['count_questions'] = teams_results_train.questionQty.apply(lambda x: sum(list(x.values())[0].values()))
teams_results_test['count_questions'] = teams_results_test.questionQty.apply(lambda x: sum(list(x.values())[0].values()))

In [28]:
teams_results_train = teams_results_train.drop(columns=['questionQty', 'locations'])
teams_results_test = teams_results_test.drop(columns=['questionQty', 'locations'])

In [29]:
columns_order = ['id_tournament', 
                 'id_team', 
                 'id_player',
                 'type_of_tournament', 
                 'duration', 
                 'count_teams',
                 'factor_name', 
                 'flag_team',
                 'count_players', 
                 'flags_players_legionnaire', 
                 'flag_player', 
                 'various_locations', 
                 'count_questions', 
                 'tournament_rate', 
                 'team_rate', 
                 'answers',
                 'position', ]

teams_results_train = teams_results_train[columns_order]
teams_results_test = teams_results_test[columns_order]

In [30]:
teams_results_train.answers = teams_results_train.answers.apply(lambda x: list(x))
teams_results_test.answers = teams_results_test.answers.apply(lambda x: list(x))

In [31]:
teams_results_train = teams_results_train.drop(index=teams_results_train.loc[teams_results_train.count_teams<3].index).reset_index(drop=True)

In [32]:
def parse_answers_by_player(df):
    columns_order = ['id_tournament', 
                     'id_team', 
                     'id_player',
                     'type_of_tournament', 
                     'duration', 
                     'count_teams',
                     'factor_name', 
                     'flag_team',
                     'count_players', 
                     'flags_players_legionnaire', 
                     'flag_player', 
                     'various_locations', 
                     'count_questions', 
                     'tournament_rate', 
                     'team_rate', 
                     'answer',
                     'id_answer',
                     'position', ]

    rows = {'id_tournament': [], 
                'id_team': [], 
                'id_player': [], 
                'type_of_tournament': [], 
                'duration': [], 
                'count_teams': [], 
                'factor_name': [], 
                'flag_team': [], 
                'count_players': [], 
                'flags_players_legionnaire': [], 
                'flag_player': [], 
                'various_locations': [], 
                'count_questions': [],
                'tournament_rate': [], 
                'team_rate': [],
                'answer': [],
                'id_answer': [],
                'position': []}

    for index_row, answers in tqdm(df.to_dict()['answers'].items()):
        
        len_answers = len(answers)
        rows['id_tournament'].extend([df['id_tournament'][index_row]]*len_answers)
        rows['id_team'].extend([df['id_team'][index_row]]*len_answers)
        rows['id_player'].extend([df['id_player'][index_row]]*len_answers)
        rows['type_of_tournament'].extend([df['type_of_tournament'][index_row]]*len_answers)
        rows['duration'].extend([df['duration'][index_row]]*len_answers)
        rows['count_teams'].extend([df['count_teams'][index_row]]*len_answers)
        rows['factor_name'].extend([df['factor_name'][index_row]]*len_answers)
        rows['flag_team'].extend([df['flag_team'][index_row]]*len_answers)
        rows['count_players'].extend([df['count_players'][index_row]]*len_answers)
        rows['flags_players_legionnaire'].extend([df['flags_players_legionnaire'][index_row]]*len_answers)
        rows['flag_player'].extend([df['flag_player'][index_row]]*len_answers)
        rows['various_locations'].extend([df['various_locations'][index_row]]*len_answers)
        rows['count_questions'].extend([df['count_questions'][index_row]]*len_answers)
        rows['tournament_rate'].extend([df['tournament_rate'][index_row]]*len_answers)
        rows['team_rate'].extend([df['team_rate'][index_row]]*len_answers)
        rows['answer'].extend(answers)
        rows['id_answer'].extend(range(1, len_answers+1))
        rows['position'].extend([df['position'][index_row]]*len_answers)

    rows = pd.DataFrame(rows)
        
    return rows

In [34]:
teams_results_train = parse_answers_by_player(teams_results_train)
teams_results_test = parse_answers_by_player(teams_results_test)

100%|██████████| 112841/112841 [00:13<00:00, 8094.35it/s]


*Почистим память*

In [35]:
del results
del tournaments
gc.collect()

564562

<a class="anchor" id="baseline"></a>
[К оглавлению](#toc)\
[0. Введение](#introduction)

# 1. BaseLine модель. 
За основу базовой модели я взял логистическую регрессию.\
Основная сложность в том, что на 20млн строк повопросных данных у нас слишком мало признаков и слишком маленькое разнообразие для каждого признака. Модель  на таких данных строила близко к случайному результаты предсказаний, но различия часто были на уровне игроков, а в дальнейшем при выводеа результатов команд модель часто предсказывала слишком большое количество одинаковых позиция для команд на турнирах.

Основные показатели, которые влияют на игру команды - это сила каждого игрока в составе команды и сложность каждого вопроса для каждого игрока. Нам эти данные не известны, да и в целом сложно их как то оценить в жизни.\
Для построения модели я выбрал следующий путь:
1. __Создадим 2 признака для каждой пары игрок-вопрос.__ Признаки отвечают за __силу игрока и сложность вопроса__ для игрока. Эти признаки не отражают действительность как таковую, потому что семплирую я их случаным образом из нормального распределения, не привязываясь к чему либо. Можно считать что это "псевдосила" и "псевдосложность".
2. __Предскажим повопросные ответы игроков.__
3. __Изменим показтель силы и сложности вопроса__ для игроков так, __чтобы метрика качества предсказания увеличилась.__ При этом я сделал предположения, как будто имею дело с настоящими силами игроков и сложностью вопроса. Подробнее о калибровке ниже.
4. По достижению заданной точности я имею среднее занчение "псевдосилы" для каждого игрока и сложность вопроса с некоторой дисперсией. Эта дисперсия в сложности вопроса обеспечивала высокий результат предсказания на тренировочных данных.
5. Так как нам неизвестны данные ответов команд на турнирах за 2020 год(только для проверки качества) я не могу сделать такую калибровку на данных за 2020 год. Но я делаю предположение, что игрок, обладая определенной силой, с ней же перешел из 2019 в 2020. Поэтому __признак силы я перенес на 2020 год__, а недостающие данные игроков, которые не принимали участие в 2019 году предсказал, правда с не очень большой точностью(r2 ~ 0,3 на cv).
6. __Так как основным показателем для повопросной точности предсказания была сложность вопроса__ мне нужно где то взять ее для турниров на 2020 год. При этом, мне по сути не важна точность, что игрок, например, ответил на 2 вопрос и не ответил на 3. Если игрок ответил на 20 вопросов из 30, нам важно предсказать, что он ответил на 20 каких то вопросов из 30, а каких именно - не важно. Поэтому для сложности вопроса я сделаю следующее: __для тренировочного и тестового наборов по исходным данным при включении "псевдосилы" я предскажу сложность вопроса, предварительно обучившись на трейне, а затем добавлю дисперсию, которая будет дисперсией в целом для команды по предсказанным данным.__
7. Так как я предполагаю, что те самые случайные величины "псевдосилы" и "псевдосложности", которые я получу после всех предсказаний будут в какой то степени из одного и того же распределения что и _идеальные_ "псевдосила" и "псевдосложность", я __обучу модель логистической регрессии__ на _идеальных_ данных и предскажу результаты игрок-вопрос для турниров 2019 и 2020 годов.

Точность повопросных результатов составила 60% на 2019 год по предсказанным сложностям вопроса без добавления шума и 44 с добавлением.\
После группировки результатов модели в число правильных ответов для комнды(порог - квантиль 75% правильных ответов от игроков) я определил командам метса по числу правильных ответов на конкретном турнире.\
Затем тоже самое я проделал для данных за 2020 год.

__Результаты BaseLine модели:__\
___2019 год:___\
__Корреляция Спирмена: 0.898\
Корреляция Кендалла: 0.735\
Повопросный F1: 0.44__\
___2020 год:___\
__Корреляция Спирмена: 0.822\
Корреляция Кендалла: 0.644\
Повопросный F1: 0.41__

[К оглавлению](#toc)\
[2. EM Алгоритм](#em)

In [56]:
teams_results_train['power_of_player'] = np.random.normal(200, 10, teams_results_train.shape[0])

In [57]:
teams_results_train['complexity'] = np.random.normal(1, 0.1, teams_results_train.shape[0])

In [58]:
teams_results_train = teams_results_train.astype('float32')
teams_results_test = teams_results_test.astype('float32')

In [43]:
X_train = teams_results_train.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position']).to_numpy()
X_train = MinMaxScaler().fit_transform(X_train)
y_train = teams_results_train['answer']

In [44]:
lr = LogisticRegression(n_jobs=-1)
inds=random.sample(range(X_train.shape[0]), X_train.shape[0]//5)
%time lr.fit(X_train[inds], y_train[inds])

y_pred = lr.predict(X_train)

accuracy = accuracy_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred, average='weighted')

print('Accuracy: ', accuracy)
print('F1-Score: ', f1)

CPU times: user 3.23 s, sys: 630 ms, total: 3.86 s
Wall time: 1min 1s
Accuracy:  0.5845360771289734
F1-Score:  0.5385157564828692


Суть калибровки в следующем:
1. Берем игрока из выборки и все его настоящие ответы за 2019 год.
2. Его сила на данный момент - среднее из того, что мы случайно для него насемплировали.
3. Если игрок ответил правильно, то ему потенцаильно начисляется 10 очков к силе, помноженные на коеффициенты, а сложность вопроса для него уменьшается в 1,05, он же верно ответил. Если игрок ответил не верно, то у него минусуется 5 очков силы, помноженные на коеффициенты, а сложность вопроса увеличивается в 1,05 раз.
4. Основной коеффициент состоит из типа турнира (0,5 или 2/3 или 1), фактора имени команды и так далее.
5. Еще один коеффициент имеет следующую логику:
 - Если игрок имеет силу выше базовых 200 и при этом отвечает не правильно, то его штраф умножается еще на $ 1 +  \frac{1}{|1-log|\frac{power}{200}||} $
 - Если игрок имеет силу меньше базовых 200 и при этом отвечает правильно, то его бонус к силе умножается еще на $ 1 +  \frac{1}{|1-log|\frac{power}{200}||} $
 - Если игрок имеет силу меньше базовых 200 и при этом отвечает неправильно, то его штраф умножается еще на $ \frac{1}{|1-log|\frac{power}{200}||} $ тем самым начисляя штраф не так сильно, ведь для слабого игрока мы ожидали скорее неправильного ответа и наказывать сильно его не нужно
6. Для каждого ответа мы складываем его прирост к силе(в + или -) в список, а после прохода всех вопросов усредняем и складываем это с текущим значением силы.
7. Так делаем для каждого игрока из основной выборки.

In [59]:
player_indices = teams_results_train.groupby('id_player', sort=False).indices

In [60]:
means_params = teams_results_train.groupby('id_player', sort=False)['type_of_tournament', 
                                         'factor_name', 
                                         'flag_team', 
                                         'flags_players_legionnaire', 
                                         'flag_player'].mean().reset_index()

In [61]:
list_answers = teams_results_train.groupby('id_player', sort=False)['answer'].agg(lambda x: x.tolist()).reset_index()

In [62]:
%%time
accuracy = 0
f1 = 0
count = 0
y_train = teams_results_train['answer']

while accuracy < 0.98 and f1 < 0.98:
    X_train = teams_results_train.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position']).to_numpy()
    X_train = MinMaxScaler().fit_transform(X_train)

    lr = LogisticRegression(n_jobs=-1)
    inds=random.sample(range(X_train.shape[0]), X_train.shape[0]//5)
    %time lr.fit(X_train[inds], y_train[inds])

    y_pred = lr.predict(X_train)

    accuracy = accuracy_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred, average='weighted')
    
    means_power = teams_results_train.groupby('id_player', sort=False)['power_of_player'].mean().reset_index()
    list_complexity = teams_results_train.groupby('id_player', sort=False)['complexity'].agg(lambda x: x.tolist()).reset_index()
    
    if accuracy < 0.98 and f1 < 0.98:
        count += 1 
        print('Номер текущей итерации: ', count)
        print('Accuracy: ', accuracy)
        print('F1-Score: ', f1)
        
        for id_player in tqdm(teams_results_train.id_player.unique()):
            power = means_power[means_power.id_player==id_player]['power_of_player'].values[0]
            answers = list_answers[list_answers.id_player==id_player]['answer'].values[0]
            complexity = list_complexity[list_complexity.id_player==id_player]['complexity'].values[0]

            type_of_tournament = means_params[means_params.id_player==id_player]['type_of_tournament'].values[0]
            factor_name = means_params[means_params.id_player==id_player]['factor_name'].values[0]
            flag_team = means_params[means_params.id_player==id_player]['flag_team'].values[0]
            flags_players_legionnaire = means_params[means_params.id_player==id_player]['flags_players_legionnaire'].values[0]
            flag_player = means_params[means_params.id_player==id_player]['flag_player'].values[0]

            main_koef = type_of_tournament*factor_name*flag_team*flags_players_legionnaire*flag_player

            replace_complexity = {1: 0.95, 0: 1.05}
            complexity = np.vectorize(lambda x: replace_complexity[x])(answers)*complexity

            replace_answer = {1: 10, 0: -5}
            answers = np.vectorize(lambda x: replace_answer[x])(answers)

            if power > 200:
                replace_koef_power = {10: 1, -5: 1+1/abs(1-np.log(abs(power/200)))}
            else:
                replace_koef_power = {10: 1+1/abs(1-np.log(abs(power/200))), -5: 1/abs(1-np.log(abs(power/200)))}

            koef_power = np.vectorize(lambda x: replace_koef_power[x])(answers)
                        
            power += np.mean(answers*complexity*koef_power)*main_koef
            
            ids_player = player_indices[id_player]
            
            power = np.array([power]*ids_player.shape[0])
            
            teams_results_train.at[ids_player, 'power_of_player'] = power
            teams_results_train.at[ids_player, 'complexity'] = complexity
        
        
    else:
        break
    
    
print('Итерация до заданной точности: ', count)
print('Accuracy: ', accuracy)
print('F1-Score: ', f1)

CPU times: user 2.27 s, sys: 1.05 s, total: 3.32 s
Wall time: 50.4 s


  0%|          | 0/59059 [00:00<?, ?it/s]

Номер текущей итерации:  0
Accuracy:  0.5846303654189727
F1-Score:  0.5384364502422636


100%|██████████| 59059/59059 [07:10<00:00, 137.27it/s]


CPU times: user 4.69 s, sys: 1.26 s, total: 5.94 s
Wall time: 1min 6s


...

CPU times: user 5.31 s, sys: 1.29 s, total: 6.61 s
Wall time: 1min 22s


  0%|          | 0/59059 [00:00<?, ?it/s]

Номер текущей итерации:  4
Accuracy:  0.9768639868372779
F1-Score:  0.9768736351347707


100%|██████████| 59059/59059 [07:11<00:00, 136.98it/s]


CPU times: user 4.75 s, sys: 1.32 s, total: 6.07 s
Wall time: 59.9 s
Итерация до заданной точности:  5
Accuracy:  0.9931988089718284
F1-Score:  0.9932000832554502
CPU times: user 51min 45s, sys: 6min 50s, total: 58min 35s
Wall time: 44min 41s


In [3]:
print('Средняя сила всех игроков - %f'%(teams_results_train.power_of_player.mean()))
print('Разброс силы всех игроков - %f'%(teams_results_train.power_of_player.std()))

Средняя сила всех игроков - 206.194061
Разброс силы всех игроков - 7.164006


In [4]:
print('Средняя сложность вопроса - %f'%(teams_results_train.complexity.mean()))
print('Разброс сложности вопросов - %f'%(teams_results_train.complexity.std()))

Средняя сложность вопроса - 1.094619
Разброс сложности вопросов - 0.270276


In [5]:
ankwnown_power_of_players = list(set(teams_results_test.id_player.unique()).difference(set(teams_results_train.id_player.unique())))

In [6]:
train_for_predict_power = teams_results_train.groupby(['id_player'], sort=False)['type_of_tournament', 
                                                       'duration', 
                                                       'count_teams', 
                                                       'factor_name', 
                                                       'flag_team', 
                                                       'count_players',
                                                       'flags_players_legionnaire', 
                                                       'flag_player', 
                                                       'various_locations',
                                                       'count_questions', 
                                                       'tournament_rate', 
                                                       'team_rate', 
                                                       'power_of_player'].mean()
train_for_predict_power.reset_index(inplace=True)

In [7]:
train_id_players = random.sample(train_for_predict_power.id_player.unique().tolist(), 50000)
val_id_players = list(set(train_for_predict_power.id_player.unique()).difference(set(train_id_players)))

In [8]:
X_train = MinMaxScaler().fit_transform(train_for_predict_power.loc[train_for_predict_power.id_player.isin(train_id_players)].iloc[:,1:-1])
y_train = train_for_predict_power.loc[train_for_predict_power.id_player.isin(train_id_players), 'power_of_player']

X_val = MinMaxScaler().fit_transform(train_for_predict_power.loc[train_for_predict_power.id_player.isin(val_id_players)].iloc[:,1:-1])
y_val = train_for_predict_power.loc[train_for_predict_power.id_player.isin(val_id_players), 'power_of_player']

In [29]:
xgb_r_best = XGBRegressor(n_estimators=2000, max_depth=12, learning_rate=0.05, tree_method='gpu_hist', gpu_id=0, random_state=42)
xgb_r_best.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=12,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=2000, n_jobs=16, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [30]:
train_pred = xgb_r_best.predict(X_train)
val_pred = xgb_r_best.predict(X_val)

In [31]:
print('Train')
print('R2: %f'%(r2_score(y_train, train_pred)))
print('MAE: %f'%(mean_absolute_error(y_train, train_pred)))
print('MSE: %f'%(np.sqrt(mean_squared_error(y_train, train_pred))))

Train
R2: 0.783061
MAE: 1.116039
MSE: 2.224652


In [32]:
print('Val')
print('R2: %f'%(r2_score(y_val, val_pred)))
print('MAE: %f'%(mean_absolute_error(y_val, val_pred)))
print('MSE: %f'%(np.sqrt(mean_squared_error(y_val, val_pred))))

Val
R2: 0.458603
MAE: 2.467639
MSE: 3.530744


Теперь применим эту модель для прогнозирования недостающей силы игроков за 2020 год

In [33]:
teams_results_test['power_of_player'] = 0

In [34]:
test_for_predict_power = teams_results_test.groupby(['id_player'], sort=False)['type_of_tournament', 
                                                       'duration', 
                                                       'count_teams', 
                                                       'factor_name', 
                                                       'flag_team', 
                                                       'count_players',
                                                       'flags_players_legionnaire', 
                                                       'flag_player', 
                                                       'various_locations',
                                                       'count_questions', 
                                                       'tournament_rate', 
                                                       'team_rate', 
                                                       'power_of_player'].mean()
test_for_predict_power.reset_index(inplace=True)

In [35]:
for id_player in tqdm(teams_results_test.id_player.unique()):
    if id_player in train_for_predict_power.id_player.unique():
        teams_results_test.loc[teams_results_test.id_player==id_player, 'power_of_player'] = \
            train_for_predict_power.loc[train_for_predict_power.id_player==id_player, 'power_of_player'].mean()
    else:
        player_info = test_for_predict_power[test_for_predict_power.id_player==id_player]
        power_player = xgb_r_best.predict(MinMaxScaler().fit_transform(player_info.iloc[:,1:-1]))
        
        teams_results_test.loc[teams_results_test.id_player==id_player, 'power_of_player'] = power_player[0]
            

100%|██████████| 28996/28996 [08:57<00:00, 53.93it/s]


In [36]:
print('Средняя сила всех игроков за 2020 - %f'%(teams_results_test.power_of_player.mean()))
print('Разброс силы всех игроков за 2020 - %f'%(teams_results_test.power_of_player.std()))

Средняя сила всех игроков за 2020 - 200.003019
Разброс силы всех игроков за 2020 - 2.992994


### 2019 BaseLine

In [38]:
train_for_predict_complexity = teams_results_train.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['type_of_tournament', 
                                                       'duration', 
                                                       'count_teams', 
                                                       'factor_name', 
                                                       'flag_team', 
                                                       'count_players',
                                                       'flags_players_legionnaire', 
                                                       'flag_player', 
                                                       'various_locations',
                                                       'count_questions', 
                                                       'tournament_rate', 
                                                       'team_rate', 
                                                       'power_of_player',
                                                       'complexity',].mean()
train_for_predict_complexity.reset_index(inplace=True)

train_id_players = random.sample(train_for_predict_power.id_player.unique().tolist(), 50000)
val_id_players = list(set(train_for_predict_power.id_player.unique()).difference(set(train_id_players)))

In [39]:
X_train = MinMaxScaler().fit_transform(train_for_predict_complexity.loc[train_for_predict_complexity.id_player.isin(train_id_players)].iloc[:,3:-1])
y_train = train_for_predict_complexity.loc[train_for_predict_complexity.id_player.isin(train_id_players), 'complexity']

X_val = MinMaxScaler().fit_transform(train_for_predict_complexity.loc[train_for_predict_complexity.id_player.isin(val_id_players)].iloc[:,3:-1])
y_val = train_for_predict_complexity.loc[train_for_predict_complexity.id_player.isin(val_id_players), 'complexity']

In [52]:
xgb_r_best = XGBRegressor(n_estimators=1000, max_depth=15, learning_rate=0.05, tree_method='gpu_hist', gpu_id=0, random_state=42)
xgb_r_best.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=16, num_parallel_tree=1, random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='gpu_hist', validate_parameters=1, verbosity=None)

In [53]:
train_pred = xgb_r_best.predict(X_train)
val_pred = xgb_r_best.predict(X_val)

In [54]:
print('Train')
print('R2: %f'%(r2_score(y_train, train_pred)))
print('MAE: %f'%(mean_absolute_error(y_train, train_pred)))
print('MSE: %f'%(np.sqrt(mean_squared_error(y_train, train_pred))))

Train
R2: 0.962118
MAE: 0.012590
MSE: 0.018955


In [55]:
print('Val')
print('R2: %f'%(r2_score(y_val, val_pred)))
print('MAE: %f'%(mean_absolute_error(y_val, val_pred)))
print('MSE: %f'%(np.sqrt(mean_squared_error(y_val, val_pred))))

Val
R2: 0.605293
MAE: 0.047286
MSE: 0.060897


In [56]:
del train_for_predict_power
del test_for_predict_power

In [61]:
X = MinMaxScaler().fit_transform(train_for_predict_complexity.iloc[:,3:-1])
complexity_pred = xgb_r_best.predict(X)

In [62]:
train_for_predict_complexity['complexity_pred'] = complexity_pred

In [63]:
teams_results_train = pd.merge(teams_results_train, train_for_predict_complexity[['id_tournament', 'id_team', 'id_player', 'complexity_pred']], on=['id_tournament', 'id_team', 'id_player'])

In [65]:
del X

In [67]:
lr = LogisticRegression(n_jobs=-1)

In [78]:
X_true = teams_results_train.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position', 'complexity_pred']).to_numpy()
X_true = MinMaxScaler().fit_transform(X_true)
y_true = teams_results_train['answer']

inds=random.sample(range(X_true.shape[0]), X_true.shape[0]//2)
%time lr.fit(X_true[inds], y_true[inds])

CPU times: user 18.7 s, sys: 1.03 s, total: 19.7 s
Wall time: 2min 45s


LogisticRegression(n_jobs=-1)

In [79]:
X_train = teams_results_train.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position', 'complexity']).to_numpy()
X_train = MinMaxScaler().fit_transform(X_train)

In [83]:
f1_score(y_true, lr.predict(X_train), average='weighted')

0.5986750372822552

In [84]:
std_complexity_pred = teams_results_train.groupby(['id_tournament', 'id_team'], sort=False)['complexity_pred'].agg(lambda x: np.sqrt(x.std())/2)

In [86]:
inds_tournament_team = teams_results_train.groupby(['id_tournament', 'id_team'], sort=False).indices

In [90]:
for inds, std_complexity in tqdm(std_complexity_pred.to_dict().items()):
    inds = inds_tournament_team[inds]
    complexity_for_tournament = teams_results_train.iloc[inds]['complexity_pred'].values
    noise_complexity = np.random.normal(0, std_complexity, len(inds))
    complexity = complexity_for_tournament + noise_complexity
    
    teams_results_train.at[inds, 'complexity_pred'] = complexity

100%|██████████| 86360/86360 [01:31<00:00, 947.99it/s] 


In [91]:
X_train = teams_results_train.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position', 'complexity']).to_numpy()
X_train = MinMaxScaler().fit_transform(X_train)

y_train = teams_results_train['answer']

In [92]:
f1_score(y_train, lr.predict(X_train), average='weighted')

0.4426680776713311

In [94]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                             _:  2.4 GiB
           teams_results_train:  2.4 GiB
                           _64:  2.4 GiB
                           _93:  2.4 GiB
                       X_train:  1.1 GiB
                        X_true:  1.1 GiB
                       y_train: 878.4 MiB
                        y_true: 878.4 MiB
            teams_results_test: 340.2 MiB
  train_for_predict_complexity: 36.2 MiB


In [95]:
teams_results_train['answer_pred'] = lr.predict(X_train)

In [96]:
true_position = pd.DataFrame(teams_results_train.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position = true_position.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [97]:
pred_position = teams_results_train.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred'].sum()
pred_position = pred_position.reset_index()
pred_position = pred_position.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred'].agg(lambda x: x.quantile(0.75))
pred_position = pred_position.reset_index()    
pred_position = pred_position.sort_values(by=['id_tournament', 'answer_pred'], ascending=False).reset_index()

In [98]:
for id_tournament in tqdm(pred_position.id_tournament.unique()):
    answers_pred = pred_position.loc[pred_position.id_tournament==id_tournament, 'answer_pred'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position.loc[pred_position.id_tournament==id_tournament, 'position_pred'] = positions
    

100%|██████████| 672/672 [00:07<00:00, 86.09it/s] 


In [99]:
pred_position = pred_position.sort_values(by=['id_tournament', 'id_team'])

In [106]:
predict_positions_result = pd.merge(true_position, pred_position, on=['id_tournament','id_team'])
predict_positions_result = predict_positions_result.reset_index()

In [135]:
corr_spearman, _ = spearmanr(predict_positions_result['position'], predict_positions_result['position_pred'])
corr_kendall, _ = kendalltau(predict_positions_result['position'], predict_positions_result['position_pred'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.898
Корреляция Кендалла: 0.735


### 2020 BaseLine

In [108]:
test_for_predict_complexity = teams_results_test.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['type_of_tournament', 
                                                       'duration', 
                                                       'count_teams', 
                                                       'factor_name', 
                                                       'flag_team', 
                                                       'count_players',
                                                       'flags_players_legionnaire', 
                                                       'flag_player', 
                                                       'various_locations',
                                                       'count_questions', 
                                                       'tournament_rate', 
                                                       'team_rate', 
                                                       'power_of_player'].mean()
test_for_predict_complexity.reset_index(inplace=True)

In [110]:
X = MinMaxScaler().fit_transform(test_for_predict_complexity.iloc[:,3:])

In [111]:
complexity_pred = xgb_r_best.predict(X)

In [112]:
test_for_predict_complexity['complexity_pred'] = complexity_pred

In [113]:
teams_results_test = pd.merge(teams_results_test, test_for_predict_complexity[['id_tournament', 'id_team', 'id_player', 'complexity_pred']], on=['id_tournament', 'id_team', 'id_player'])

In [114]:
std_complexity_pred = teams_results_test.groupby(['id_tournament', 'id_team'], sort=False)['complexity_pred'].agg(lambda x: np.sqrt(x.std())/2)

In [115]:
inds_tournament_team_test = teams_results_test.groupby(['id_tournament', 'id_team'], sort=False).indices

In [116]:
for inds, std_complexity in tqdm(std_complexity_pred.to_dict().items()):
    inds = inds_tournament_team_test[inds]
    complexity_for_tournament = teams_results_test.iloc[inds]['complexity_pred'].values
    noise_complexity = np.random.normal(0, std_complexity, len(inds))
    complexity = complexity_for_tournament + noise_complexity
    
    teams_results_test.at[inds, 'complexity_pred'] = complexity

100%|██████████| 22366/22366 [03:36<00:00, 103.47it/s]


In [117]:
X_test = teams_results_test.drop(columns=['id_tournament', 'id_team', 'id_player', 'answer', 'id_answer', 'position']).to_numpy()
X_test = MinMaxScaler().fit_transform(X_test)

In [118]:
teams_results_test['answer_pred'] = lr.predict(X_test)

In [119]:
f1_score(teams_results_test['answer'], teams_results_test['answer_pred'], average='weighted')

0.4150792134603233

In [121]:
true_position_test = pd.DataFrame(teams_results_test.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position_test = true_position_test.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [122]:
pred_position_test = teams_results_test.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred'].sum()
pred_position_test = pred_position_test.reset_index()
pred_position_test = pred_position_test.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred'].agg(lambda x: x.quantile(0.75))
pred_position_test = pred_position_test.reset_index()    
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'answer_pred'], ascending=False).reset_index()

In [123]:
for id_tournament in tqdm(pred_position_test.id_tournament.unique()):
    answers_pred = pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'answer_pred'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'position_pred'] = positions
    

100%|██████████| 173/173 [00:02<00:00, 84.38it/s] 


In [124]:
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'id_team'])

In [125]:
predict_positions_result_test = pd.merge(true_position_test, pred_position_test, on=['id_tournament','id_team'])
predict_positions_result_test = predict_positions_result_test.reset_index()

In [132]:
corr_spearman, _ = spearmanr(predict_positions_result_test['position'], predict_positions_result_test['position_pred'])
corr_kendall, _ = kendalltau(predict_positions_result_test['position'], predict_positions_result_test['position_pred'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.822
Корреляция Кендалла: 0.644


### Рейтинг игроков.
Просто взял и посчитал число правильных ответов для каждого игрока по предсказанным данным.\
p.s. Николенко Сергей где то примерно на 703 месте, с 51 правильным ответом, сорян(

In [34]:
rating_player = pd.merge(players, 
         teams_results_test.groupby('id_player', sort=False)['answer_pred'].sum(), 
         left_on='id', 
         right_on='id_player', 
         sort=False).reset_index()\
[['surname', 
    'name', 
    'answer_pred']].sort_values('answer_pred', 
                                ascending=False).reset_index()
rating_player.head(20)

,index,surname,name,answer_pred
0,2981,Сорожкин,Артём,556.0
1,366,Болган,Валерий,450.0
2,3126,Тимошенко,Егор,420.0
3,2713,Рыжанова,Наталия,403.0
4,1206,Иванченко,Сергей,398.0
5,3196,Тюнькин,Игорь,393.0
6,924,Дидбаридзе,Юлия,379.0
7,2055,Мельникова,Ольга,374.0
8,8333,Черкасов,Сергей,369.0
9,2996,Спешков,Сергей,369.0


<a class="anchor" id="em"></a>
[К оглавлению](#toc)\
[1. BaseLine модель](#baseline)

# 2. EM Алгоритм

Когда игрок сидит за столом на правильность его мыслей и, как результат, на качество его игры влияют сокомандники.\
Бывает такое, что игрок единственный за столом знает ответ, а вся остальная команда думает неправильно и игрок под действием мнения большинства не "топит" за свою версию ответа, а команда в итоге дает неверный ответ.\
С другой стороны, когда игрок единственный думает неправильно, а все остальные в его команде рассуждают правильно, то и он скорее всего примет их точку зрения и команда в итоге ответит правильно.\
Бывают и исключения, а бывает и такая схема взаимодействия игроков в команде, когда у каждого игрока есть роль и, в зависимости от тематики вопроса, даже один в поле воин, точнее другие уверены в его абсолютном знании предметной области по сравнению с остальной командой и всецело ему доверяют.\
Например, если вопрос про кинематограф и в команде есть знаток этой сферы, то, даже если команда будет сомневаться в его ответе, он может затопить и в итоге его ответ будет принят, даже если он будет неверный.\
Такая схема команды часто бывает на квизах, но на ЧГК думаю немного иначе, вопросы там скорее носят более общий характер и чаще команда в полном составе участвует в обсуждении, а решение принимает капитан или игрок взявший на себя роль лидера(на квизах - тот кто громче говорит).\
Поэтому следует учесть влияние на игрока его сокомандников в общем случае. Команда в лице игрока отвечает правильно если он сам или кто-то из его сокомандников дает правильный ответ, так как я верю, что, услышав правльный ответ во время обсуждения от одного из игроков, оставшаяся команда прислушается к нему и возьмет вопрос.

ЕМ алгоритм я буду использовать для того, чтобы учитывать правильность ответов сокомандников для каждого игрока.
М-шаг ничем не отличается от обычного ЕМ алгоритма, а Е-шаг должен учитывать влияние сокомандников.
Поэтому:\
$$\mathbb{E}{[z]} = \frac{\pi\rho_{player}}{1-\prod\limits_{teammates}(1-\pi\rho_{teammate})} ,$$\
где $\mathbb{E}{[z]}$ - математическое ожидание скрытой переменной $z$,\
$\pi$ - веротяность принадлежности к определенному классу скрытой переменной $z$,\
$\rho_{player}$ - логистическая регрессия(сигмоида) по данным игрока,\
$\prod\limits_{teammates}$ - произведение по всем игрокам,\
$\rho_{teammate}$ - логистическая регрессия(сигмоида) по данным сокомандника\

Так как показатель логарифма правдоподобия очень большой, критерий останова для данных на 2019 год будет изменение $logL$ менее чем на 100, а для 2020 года менее чем на 50 или превышение $logL$ отметки в 27 млн(без этого условия алгоритм блуждал в районе 20 млн поднимаясь в пике до 36 и затем падая до 15, приближаясь к сходимости в районе 22, когда изменение $logL$ было менее 1 тыс, но затем снова скачок).\
В общих чертах алгоритм такой: 
- Е-шаг - считаем матожидание $z$ для выборки, пересчитываем его с учетом зависимостей с определенными ограничениями, чтобы не нарваться на ошибку
- М-шаг - пересчитываем $\pi, \mu, \sigma$

Выход ЕМ алгоритма как и в BaseLine я преобразовал в уверенность(квантиль 75%) команды ответить правильно на вопрос, затем ранжировал команды потурнирно.\
__Результаты ЕМ алгоритма:__\
___2019 год:___\
__Корреляция Спирмена: 0.819\
Корреляция Кендалла: 0.636__\
___2020 год:___\
__Корреляция Спирмена: 0.797\
Корреляция Кендалла: 0.595__

[К оглавлению](#toc)\
[3. EM алгоритм v.2](#em2)

### ЕМ алгоритм 2019

In [3]:
columns = ['id_tournament', 'id_team', 'id_player', 'id_answer',
            'flag_player', 'tournament_rate', 'team_rate', 
           'power_of_player', 'complexity_pred', 'answer']
X = teams_results_train[columns]

In [4]:
indices_question_by_team = list(X.groupby(['id_tournament', 'id_team', 'id_answer'], sort=False).indices.values())

In [137]:
Xs = MinMaxScaler().fit_transform(X.iloc[:,4:-1].to_numpy())

In [208]:
def main_e_step(Xs, pis, mus, sigmas, indices, allow_singular=False):
    k = mus.shape[0]
    z = np.array([pis[i] * sp.stats.multivariate_normal.pdf(Xs, mean=mus[i], cov=sigmas[i], allow_singular=allow_singular) for i in range(k) ])   
    z = np.divide(z, np.sum(z, axis=0)).T
    z_dependent = np.array([[None, None] for _ in range(Xs.shape[0])])
    
    for team_q in tqdm(indices):
        for player in team_q:
            teammate = [_id for _id in team_q if _id != player]
            z_teammate = z[teammate]
            z_teammate = 1 - z_teammate
            z_teammate = np.prod(z_teammate, axis=0)
            z_teammate = 1 - z_teammate
            
            if z_teammate[1]<0.5**(len(team_q)-1) and z[player][1]<0.5:
                z_dependent_player = [1, 0]
            elif z_teammate[0]<0.5**(len(team_q)-1) and z[player][0]<0.5:
                z_dependent_player = [0, 1]
            elif z_teammate[1]<0.5**(len(team_q)-1) and z[player][1]>0.5:
                z_dependent_player = z[player]
            elif z_teammate[0]<0.5**(len(team_q)-1) and z[player][0]>0.5:
                z_dependent_player = z[player]
            else:
                z_dependent_player = np.divide(z[player], z_teammate)
            z_dependent[player] = z_dependent_player
    return z_dependent

def m_step(xs, z):
    k = z.shape[1]
    pis = np.sum(z, axis=0) / np.sum(z)
    mus = np.array([np.average(xs, weights=z[:,i], axis=0) for i in range(k)])
    sigmas = np.array([np.cov(xs.T, aweights=z[:,i]) for i in range(k)])
    return pis, mus, sigmas

def loglikelihood(xs, pis, mus, sigmas, allow_singular=False):
    k = mus.shape[0]
    return np.sum(np.log(np.sum(np.array([ pis[i] * sp.stats.multivariate_normal.pdf(xs, mean=mus[i], cov=sigmas[i], allow_singular=allow_singular) for i in range(k) ]), axis=0)))

In [149]:
k = 2
mus = Xs[np.random.choice(Xs.shape[0], size=k, replace=False), : ]
sigmas = np.array( [ np.identity(Xs.shape[1]) for _ in range(k) ] )
pis = (1./k) * np.ones(k)

z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team)
new_pis, new_mus, new_sigmas = m_step(Xs, z)

100%|██████████| 3789178/3789178 [10:28<00:00, 6027.30it/s] 


In [150]:
new_logl = loglikelihood(Xs, pis, mus, sigmas)
new_logl

-97477480.7705077

In [151]:
for iIter in range(5000):
    old_logl = new_logl
    z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team)
    new_pis, new_mus, new_sigmas = m_step(Xs, z)
    
    pis, mus, sigmas = new_pis, new_mus, new_sigmas
    new_logl = loglikelihood(Xs, pis, mus, sigmas)
    
    print("Логарифм правдоподобия на итерации %03d: %.6f" % (iIter, new_logl) )
    if np.abs(new_logl - old_logl) < 100:
        break

100%|██████████| 3789178/3789178 [10:32<00:00, 5991.69it/s] 


Логарифм правдоподобия на итерации 000: 90732353.644539


100%|██████████| 3789178/3789178 [10:28<00:00, 6033.29it/s] 


Логарифм правдоподобия на итерации 001: 92121534.368344


100%|██████████| 3789178/3789178 [10:31<00:00, 6003.10it/s] 


...

Логарифм правдоподобия на итерации 032: 112445284.392012


100%|██████████| 3789178/3789178 [10:19<00:00, 6116.00it/s] 


Логарифм правдоподобия на итерации 033: 112445251.397628


  1%|▏         | 50627/3789178 [00:08<09:54, 6283.43it/s]


KeyboardInterrupt: 

In [164]:
answer_pred_EM = np.argmax(z, axis=1)

In [166]:
teams_results_train['answer_pred_EM'] = answer_pred_EM

In [167]:
true_position = pd.DataFrame(teams_results_train.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position = true_position.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [168]:
pred_position = teams_results_train.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred_EM'].sum()
pred_position = pred_position.reset_index()
pred_position = pred_position.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred_EM'].agg(lambda x: x.quantile(0.75))
pred_position = pred_position.reset_index()    
pred_position = pred_position.sort_values(by=['id_tournament', 'answer_pred_EM'], ascending=False).reset_index()

In [169]:
for id_tournament in tqdm(pred_position.id_tournament.unique()):
    answers_pred = pred_position.loc[pred_position.id_tournament==id_tournament, 'answer_pred_EM'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position.loc[pred_position.id_tournament==id_tournament, 'position_pred_EM'] = positions
    

100%|██████████| 672/672 [00:01<00:00, 462.89it/s]


In [170]:
pred_position = pred_position.sort_values(by=['id_tournament', 'id_team'])

In [171]:
predict_positions_result = pd.merge(true_position, pred_position, on=['id_tournament','id_team'])
predict_positions_result = predict_positions_result.reset_index()

In [172]:
corr_spearman, _ = spearmanr(predict_positions_result['position'], predict_positions_result['position_pred_EM'])
corr_kendall, _ = kendalltau(predict_positions_result['position'], predict_positions_result['position_pred_EM'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.819
Корреляция Кендалла: 0.636


## EM алгоритм 2020

In [173]:
columns = ['id_tournament', 'id_team', 'id_player', 'id_answer',
            'flag_player', 'tournament_rate', 'team_rate', 
           'power_of_player', 'complexity_pred', 'answer']
X = teams_results_test[columns]

In [174]:
indices_question_by_team = list(X.groupby(['id_tournament', 'id_team', 'id_answer'], sort=False).indices.values())

In [175]:
Xs = MinMaxScaler().fit_transform(X.iloc[:,4:-1].to_numpy())

In [217]:
k = 2
mus = Xs[np.random.choice(Xs.shape[0], size=k, replace=False), : ]
sigmas = np.array( [ np.identity(Xs.shape[1]) for _ in range(k) ] )
pis = (1./k) * np.ones(k)

z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team)
new_pis, new_mus, new_sigmas = m_step(Xs, z)

100%|██████████| 871791/871791 [02:13<00:00, 6537.61it/s] 


In [218]:
new_logl = loglikelihood(Xs, pis, mus, sigmas)
new_logl

-21543225.58880319

In [219]:
for iIter in range(5000):
    try:
        old_logl = new_logl
        z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team, allow_singular=True)
        new_pis, new_mus, new_sigmas = m_step(Xs, z)

        pis, mus, sigmas = new_pis, new_mus, new_sigmas
        new_logl = loglikelihood(Xs, pis, mus, sigmas, allow_singular=True)

        print("Логарифм правдоподобия на итерации %03d: %.6f" % (iIter, new_logl) )
        if np.abs(new_logl - old_logl) < 50 or new_logl > 27000000:
            break
    except np.linalg.LinAlgError:
        print('Обнаружена сингулярная матрица.')
        break

100%|██████████| 871791/871791 [02:14<00:00, 6491.85it/s] 


Логарифм правдоподобия на итерации 000: 18508281.704922


100%|██████████| 871791/871791 [02:15<00:00, 6417.23it/s] 


Логарифм правдоподобия на итерации 001: 19702189.579712


100%|██████████| 871791/871791 [02:15<00:00, 6435.24it/s] 


...

Логарифм правдоподобия на итерации 013: 22800183.866796


100%|██████████| 871791/871791 [02:11<00:00, 6628.73it/s] 


Логарифм правдоподобия на итерации 014: 23551358.805856


100%|██████████| 871791/871791 [02:08<00:00, 6808.18it/s] 


Логарифм правдоподобия на итерации 015: 29177822.807212


In [220]:
answer_pred_EM = np.argmax(z, axis=1)

In [221]:
teams_results_test['answer_pred_EM'] = answer_pred_EM

In [222]:
true_position_test = pd.DataFrame(teams_results_test.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position_test = true_position_test.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [223]:
pred_position_test = teams_results_test.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred_EM'].sum()
pred_position_test = pred_position_test.reset_index()
pred_position_test = pred_position_test.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred_EM'].agg(lambda x: x.quantile(0.75))
pred_position_test = pred_position_test.reset_index()    
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'answer_pred_EM'], ascending=False).reset_index()

In [224]:
for id_tournament in tqdm(pred_position_test.id_tournament.unique()):
    answers_pred = pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'answer_pred_EM'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'position_pred_EM'] = positions
    

100%|██████████| 173/173 [00:00<00:00, 558.90it/s]


In [225]:
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'id_team'])

In [226]:
predict_positions_result_test = pd.merge(true_position_test, pred_position_test, on=['id_tournament','id_team'])
predict_positions_result_test = predict_positions_result_test.reset_index()

In [227]:
corr_spearman, _ = spearmanr(predict_positions_result_test['position'], predict_positions_result_test['position_pred_EM'])
corr_kendall, _ = kendalltau(predict_positions_result_test['position'], predict_positions_result_test['position_pred_EM'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.779
Корреляция Кендалла: 0.595


<a class="anchor" id="em2"></a>
[К оглавлению](#toc)\
[2. EM Алгоритм](#em)

# 3. EM алгоритм v.2 

Во второй версии алгоритма я решил не проходить итеративно данные за 2020 год, а применить полученные после прохода на данных 2019 года $\pi, \mu, \sigma$ к нахождению матожидания $z$, при этому алгоритм изменился в месте подсчета зависимости сокомандников на игрока: если команда ответила неверно(данные answer из датасета), то для этого игрока мы не пересчитываем вероятности, а сразу записываем ему 1 в поле $z[0]$.\
__Результаты ЕМ алгоритма v.2:__\
___2019 год:___\
__Корреляция Спирмена: 0.829\
Корреляция Кендалла: 0.654__\
___2020 год:___\
__Корреляция Спирмена: 0.780\
Корреляция Кендалла: 0.605__

[К оглавлению](#toc)\
[4. Выводы](#conclusions)

## EM алгоритм v.2 2019

In [4]:
columns = ['id_tournament', 'id_team', 'id_player', 'id_answer',
            'flag_player', 'tournament_rate', 'team_rate', 
           'power_of_player', 'complexity_pred', 'answer']
X = teams_results_train[columns]

In [5]:
indices_question_by_team = list(X.groupby(['id_tournament', 'id_team', 'id_answer'], sort=False).indices.values())

In [6]:
Xs = MinMaxScaler().fit_transform(X.iloc[:,4:-1].to_numpy())

In [7]:
true_answers = X['answer'].values

In [13]:
def main_e_step(Xs, pis, mus, sigmas, indices, true_answers, allow_singular=False):
    k = mus.shape[0]
    z = np.array([pis[i] * sp.stats.multivariate_normal.pdf(Xs, mean=mus[i], cov=sigmas[i], allow_singular=allow_singular) for i in range(k) ])   
    z = np.divide(z, np.sum(z, axis=0)).T
    z_dependent = np.array([[None, None] for _ in range(Xs.shape[0])])
    
    for team_q in tqdm(indices):
        for player in team_q:
            teammate = [_id for _id in team_q if _id != player]
            z_teammate = z[teammate]
            z_teammate = 1 - z_teammate
            z_teammate = np.prod(z_teammate, axis=0)
            z_teammate = 1 - z_teammate
            
            if true_answers[player]==0:
                z_dependent_player = [1, 0]
            elif z_teammate[1]<0.5**(len(team_q)-1) or z_teammate[0]<0.5**(len(team_q)-1):
                z_dependent_player = z[player]
            else:
                z_dependent_player = np.divide(z[player], z_teammate)
            z_dependent[player] = z_dependent_player
    z_dependent = np.divide(z_dependent, np.sum(z_dependent, axis=1)[:,np.newaxis])
    return z_dependent

def m_step(xs, z):
    k = z.shape[1]
    pis = np.sum(z, axis=0) / np.sum(z)
    mus = np.array([np.average(xs, weights=z[:,i], axis=0) for i in range(k)])
    sigmas = np.array([np.cov(xs.T, aweights=z[:,i]) for i in range(k)])
    return pis, mus, sigmas

def loglikelihood(xs, pis, mus, sigmas, allow_singular=False):
    k = mus.shape[0]
    return np.sum(np.log(np.sum(np.array([ pis[i] * sp.stats.multivariate_normal.pdf(xs, mean=mus[i], cov=sigmas[i], allow_singular=allow_singular) for i in range(k) ]), axis=0)))

In [14]:
k = 2
mus = Xs[np.random.choice(Xs.shape[0], size=k, replace=False), : ]
sigmas = np.array( [ np.identity(Xs.shape[1]) for _ in range(k) ] )
pis = (1./k) * np.ones(k)

z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team, true_answers)
new_pis, new_mus, new_sigmas = m_step(Xs, z)

100%|██████████| 3789178/3789178 [10:54<00:00, 5788.42it/s] 


In [15]:
new_logl = loglikelihood(Xs, pis, mus, sigmas)
new_logl

-98642673.9592195

In [16]:
for iIter in range(5000):
    old_logl = new_logl
    z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team, true_answers)
    new_pis, new_mus, new_sigmas = m_step(Xs, z)
    
    pis, mus, sigmas = new_pis, new_mus, new_sigmas
    new_logl = loglikelihood(Xs, pis, mus, sigmas)
    
    print("Логарифм правдоподобия на итерации %03d: %.6f" % (iIter, new_logl) )
    if np.abs(new_logl - old_logl) < 100:
        break

100%|██████████| 3789178/3789178 [10:57<00:00, 5765.23it/s] 


Логарифм правдоподобия на итерации 000: 91994522.310956


100%|██████████| 3789178/3789178 [11:00<00:00, 5738.56it/s] 


Логарифм правдоподобия на итерации 001: 96660070.297602


100%|██████████| 3789178/3789178 [10:57<00:00, 5762.24it/s] 


...

Логарифм правдоподобия на итерации 017: 99461514.108920


100%|██████████| 3789178/3789178 [10:53<00:00, 5795.62it/s] 


Логарифм правдоподобия на итерации 018: 99461323.050407


100%|██████████| 3789178/3789178 [10:52<00:00, 5809.96it/s] 


Логарифм правдоподобия на итерации 019: 99461296.816228


In [17]:
answer_pred_EM_2 = np.argmax(z, axis=1)

In [18]:
teams_results_train['answer_pred_EM_2'] = answer_pred_EM_2

In [19]:
true_position = pd.DataFrame(teams_results_train.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position = true_position.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [20]:
pred_position = teams_results_train.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred_EM_2'].sum()
pred_position = pred_position.reset_index()
pred_position = pred_position.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred_EM_2'].agg(lambda x: x.quantile(0.75))
pred_position = pred_position.reset_index()    
pred_position = pred_position.sort_values(by=['id_tournament', 'answer_pred_EM_2'], ascending=False).reset_index()

In [21]:
for id_tournament in tqdm(pred_position.id_tournament.unique()):
    answers_pred = pred_position.loc[pred_position.id_tournament==id_tournament, 'answer_pred_EM_2'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position.loc[pred_position.id_tournament==id_tournament, 'position_pred_EM_2'] = positions
    

100%|██████████| 672/672 [00:01<00:00, 354.11it/s]


In [22]:
pred_position = pred_position.sort_values(by=['id_tournament', 'id_team'])

In [23]:
predict_positions_result = pd.merge(true_position, pred_position, on=['id_tournament','id_team'])
predict_positions_result = predict_positions_result.reset_index()

In [24]:
corr_spearman, _ = spearmanr(predict_positions_result['position'], predict_positions_result['position_pred_EM_2'])
corr_kendall, _ = kendalltau(predict_positions_result['position'], predict_positions_result['position_pred_EM_2'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.829
Корреляция Кендалла: 0.654


## EM алгоритм v.2 2020

In [25]:
columns = ['id_tournament', 'id_team', 'id_player', 'id_answer',
            'flag_player', 'tournament_rate', 'team_rate', 
           'power_of_player', 'complexity_pred', 'answer']
X = teams_results_test[columns]

In [40]:
Xs = MinMaxScaler().fit_transform(X.iloc[:,4:-1].to_numpy())

In [32]:
indices_question_by_team = list(X.groupby(['id_tournament', 'id_team', 'id_answer'], sort=False).indices.values())

In [41]:
def main_e_step(Xs, pis, mus, sigmas, indices, allow_singular=False):
    k = mus.shape[0]
    z = np.array([pis[i] * sp.stats.multivariate_normal.pdf(Xs, mean=mus[i], cov=sigmas[i], allow_singular=allow_singular) for i in range(k) ])   
    z = np.divide(z, np.sum(z, axis=0)).T
    z_dependent = np.array([[None, None] for _ in range(Xs.shape[0])])
    
    for team_q in tqdm(indices):
        for player in team_q:
            teammate = [_id for _id in team_q if _id != player]
            z_teammate = z[teammate]
            z_teammate = 1 - z_teammate
            z_teammate = np.prod(z_teammate, axis=0)
            z_teammate = 1 - z_teammate
            
            if z_teammate[1]<0.5**(len(team_q)-1) or z_teammate[0]<0.5**(len(team_q)-1):
                z_dependent_player = z[player]
            else:
                z_dependent_player = np.divide(z[player], z_teammate)
            z_dependent[player] = z_dependent_player
    z_dependent = np.divide(z_dependent, np.sum(z_dependent, axis=1)[:,np.newaxis])
    return z_dependent


In [42]:
k = 2
z = main_e_step(Xs, pis, mus, sigmas, indices_question_by_team)

100%|██████████| 871791/871791 [02:07<00:00, 6845.01it/s] 


In [43]:
answer_pred_EM_2 = np.argmax(z, axis=1)

In [44]:
teams_results_test['answer_pred_EM_2'] = answer_pred_EM_2

In [45]:
true_position_test = pd.DataFrame(teams_results_test.groupby(['id_tournament', 'id_team'], sort=False)['position'].mean())
true_position_test = true_position_test.sort_values(by=['id_tournament', 'id_team']).reset_index()

In [46]:
pred_position_test = teams_results_test.groupby(['id_tournament', 'id_team', 'id_player'], sort=False)['answer', 'answer_pred_EM_2'].sum()
pred_position_test = pred_position_test.reset_index()
pred_position_test = pred_position_test.groupby(['id_tournament', 'id_team'], sort=False)['answer', 'answer_pred_EM_2'].agg(lambda x: x.quantile(0.75))
pred_position_test = pred_position_test.reset_index()    
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'answer_pred_EM_2'], ascending=False).reset_index()

In [47]:
for id_tournament in tqdm(pred_position_test.id_tournament.unique()):
    answers_pred = pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'answer_pred_EM_2'].values
    num_teams = len(answers_pred)
    positions = [0]*num_teams
    start_position = 1
    while start_position<num_teams:
        answers = answers_pred[start_position-1]
        current_answers = answers_pred.tolist().count(answers)
        if current_answers > 1:
            current_pos = start_position + current_answers/2
        else:
            current_pos = start_position
        for pos in range(current_answers):
            positions[start_position+pos-1] = current_pos
        start_position += current_answers
        
    pred_position_test.loc[pred_position_test.id_tournament==id_tournament, 'position_pred_EM_2'] = positions
    

100%|██████████| 173/173 [00:00<00:00, 465.88it/s]


In [48]:
pred_position_test = pred_position_test.sort_values(by=['id_tournament', 'id_team'])

In [49]:
predict_positions_result_test = pd.merge(true_position_test, pred_position_test, on=['id_tournament','id_team'])
predict_positions_result_test = predict_positions_result_test.reset_index()

In [50]:
corr_spearman, _ = spearmanr(predict_positions_result_test['position'], predict_positions_result_test['position_pred_EM_2'])
corr_kendall, _ = kendalltau(predict_positions_result_test['position'], predict_positions_result_test['position_pred_EM_2'])
print('Корреляция Спирмена: %.3f' % corr_spearman)
print('Корреляция Кендалла: %.3f' % corr_kendall)

Корреляция Спирмена: 0.780
Корреляция Кендалла: 0.605


[К оглавлению](#toc)\
[3. EM алгоритм v.2](#em2)

# 4. Выводы. <a class="anchor" id="conclusions"></a>
Домашняя работа очень понравилась. Сдаю позже, так как в предпоследний день еще в ноутбуке без какого либо оформления и записей мыслей, меня посетила светлая идея, не оправдавшая себя, так что ее тут нет.\
Большую часть времени в этом ДЗ я уделил BaseLine модели, как ни странно. Возможно, если бы я смог уделить ДЗ больше времени, и дотюнил бы модель ЕМ алгоритма, ее результаты были бы выше базовой. Но по факту заданные метрики качества оказались выше именно у базовой модели.

[К оглавлению](#toc)